<a href="https://colab.research.google.com/github/rsabilio/ia024-projeto-rag/blob/main/1_extracao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing packages

In [1]:
!pip install -q llama-index llama-parse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.7 MB/s eta 0:00:00


# Importing packages

In [2]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
import os
import json
import csv
import re
import pandas as pd
from google.colab import userdata
from llama_parse import LlamaParse

nest_asyncio.apply()

os.environ["LLAMA_CLOUD_API_KEY"] = userdata.get('LLAMA_CLOUD_API_KEY_VAGNER')
GITHUB_TOKEN      = userdata.get('GITHUB_TOKEN')
GITHUB_USER_EMAIL = userdata.get('GITHUB_USER_EMAIL')
GITHUB_USER_NAME  = userdata.get('GITHUB_USER_NAME')

!git config --global user.email '''{GITHUB_USER_EMAIL}'''
!git config --global user.name '''{GITHUB_USER_NAME}'''

root = "/content/ia024-projeto-rag"

# Cloning git repo

In [3]:
# Clonar o repositório GitHub
!git clone https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag.git

Cloning into 'ia024-projeto-rag'...
remote: Enumerating objects: 5929, done.
remote: Counting objects: 100% (934/934), done.
remote: Compressing objects: 100% (398/398), done.
remote: Total 5929 (delta 592), reused 856 (delta 528), pack-reused 4995
Receiving objects: 100% (5929/5929), 1.12 GiB | 23.55 MiB/s, done.
Resolving deltas: 100% (1981/1981), done.
Updating files: 100% (5839/5839), done.


In [4]:
def save_on_git(msg):

    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

    !git add .

    !git commit -m '''{msg}'''

    !git push origin main

In [8]:
def update_local_repo():
    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

In [18]:
update_local_repo()

/content/ia024-projeto-rag
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6 (delta 3), reused 6 (delta 3), pack-reused 0
Unpacking objects: 100% (6/6), 656 bytes | 328.00 KiB/s, done.
From https://github.com/rsabilio/ia024-projeto-rag
 * branch            main       -> FETCH_HEAD
   fad2937..3c7b093  main       -> origin/main
Updating fad2937..3c7b093
Fast-forward
 ...3o Did\303\241tica Educa\303\247\303\243o B\303\241sica.pdf" |  Bin 884291 -> 0 bytes
 ...d\303\241tica Educa\303\247\303\243o B\303\241sica.pdf.json" | 3372 ---------------------------
 2 files changed, 3372 deletions(-)
 delete mode 100644 "0-downloaded-files/0-reitoria/2018.08.07 - Resolu\303\247\303\243o 62 - Aprova Organiza\303\247\303\243o Did\303\241tica Educa\303\247\303\243o B\303\241sica.pdf"
 delete mode 100644 "1-files-content/0-reitoria/2018.08.07 - Resolu\303\247\303\243o 62 - Aprova Organiza\303\247\303\243o D

# Extracting to JSON

In [28]:
def extract_and_save_content(parser, src_dir, dst_dir, log_file):

    # Verifica se o arquivo de log existe
    if not os.path.exists(log_file):
        # Cria o arquivo de log com cabeçalhos se não existir
        with open(log_file, 'w', newline='') as csvfile:
            log_writer = csv.writer(csvfile)
            log_writer.writerow(['file_path', 'num_pages'])  # Cabeçalhos do CSV

    for root, dirs, files in os.walk(src_dir):
        for file in files:
            src_file_path = os.path.join(root, file)
            relative_path = os.path.relpath(root, src_dir)
            dst_file_dir  = os.path.join(dst_dir, relative_path)

            #print(src_file_path)

            # Cria o diretório de destino se não existir
            os.makedirs(dst_file_dir, exist_ok=True)

            # Caminho completo do arquivo de destino
            dst_file_path = os.path.join(dst_file_dir, file)
            dst_file_path = f"{dst_file_path}.json"
            if os.path.exists(dst_file_path):
                print(f"*** {src_file_path}  já processado")
                continue

            # Extrai o conteúdo do arquivo usando LlamaParse
            try:
                print(f"{src_file_path}")
                json_objs = parser.get_json_result(src_file_path)

                with open(dst_file_path, 'w', encoding='utf-8') as f:
                    json.dump(json_objs, f, ensure_ascii=False, indent=4)

                # Conta o número de páginas no conteúdo processado
                num_pages = len(json_objs[0]["pages"])

                # Escreve as informações do log em tempo real
                with open(log_file, 'a', newline='') as csvfile:
                    log_writer = csv.writer(csvfile)
                    log_writer.writerow([src_file_path, num_pages])

            except Exception as e:
                print(f"Erro ao processar o arquivo {src_file_path}: {e}")

update_local_repo()

# Caminhos
src_directory = f"{root}/0-downloaded-files/2-boituva"
dst_directory = f"{root}/1-files-content/2-boituva"
log_file_path = f"{root}/log-content-extraction.csv"

parser = LlamaParse(verbose=False)

# TODO: generalizar
if src_directory.endswith('portarias'):
    # Extrai e salva o conteúdo mantendo a estrutura dos diretórios
    for ano in range(2014, 2015):
        print(f"\nProcessing: {ano}")
        src_directory_year = f"{src_directory}/{ano}"
        dst_directory_year = f"{dst_directory}/{ano}"

        print("extracting...")
        extract_and_save_content(parser, src_directory_year, dst_directory_year, log_file_path)

        print("\npushing to git...")
        msg = 'Adicionando JSON das Portarias de Boituva'
        save_on_git(msg)
else:
    print(f"\nProcessing: {src_directory}")

    print("extracting...")
    extract_and_save_content(parser, src_directory, dst_directory, log_file_path)

    print("\npushing to git...")
    msg = 'Adicionando JSON das Organizações didáticas'
    save_on_git(msg)


/content/ia024-projeto-rag
From https://github.com/rsabilio/ia024-projeto-rag
 * branch            main       -> FETCH_HEAD
Already up to date.

Processing: /content/ia024-projeto-rag/0-downloaded-files/2-boituva
extracting...
*** /content/ia024-projeto-rag/0-downloaded-files/2-boituva/ppp/Resolucao 06-2016 Aprova PPP.pdf  já processado
*** /content/ia024-projeto-rag/0-downloaded-files/2-boituva/portarias/2019/03 - Março/Mar_Port_0034_Designa membros do NDE de Engenharia de produção_BTV.pdf  já processado
*** /content/ia024-projeto-rag/0-downloaded-files/2-boituva/portarias/2019/03 - Março/Mar_Port_0033_Fiscal de contrato  Segurança Patrimonial_BTV.pdf  já processado
*** /content/ia024-projeto-rag/0-downloaded-files/2-boituva/portarias/2019/03 - Março/Mar_Port_0027_Designar servidor comissão de seleção de projetos de ensino do Campus Votuporanga_BTV.pdf  já processado
*** /content/ia024-projeto-rag/0-downloaded-files/2-boituva/portarias/2019/03 - Março/Mar_Port_0030_Designar membros Eq

# Organizing dataset

## Estrutura do texto do Estatuto

* Página 1: Resolução de aprovação de alteração do estatuto

* Página 2 em diante: Texto do Estatuto

* O Estatuto possui Título, Capítulos, Seções, Subseções e Artigos.

* Um Título pode ou não ter capítulos.

* Um Capítulo pode ou não ter seções.

Considerei a estrutura até o nível de Capítulos e coloquei todo o conteúdo do capítulo (seções, subseções e artigos) como um texto só.

### Carregando JSON gerado pelo LlamaParse

In [62]:
#json_file = f"{root}/1-files-content/0-reitoria/ESTATUTO IFSP - VERSÃO VIGENTE - Resolução 08-2014 - - ALERAÇÃO da Resolução 872-2013 .pdf.json"
#json_file = f"{root}/1-files-content/0-reitoria/2016.12.06 - Resolução 147 - Aprova Organização Didática Cursos Graduação (Pesquisável).pdf.json"
json_file = f"{root}/1-files-content/0-reitoria/2022.09.06-Resolução-Normativa-N.10-2022-Altera-Resol.-62-2018-O.D.-Ed.-Básica.pdf.json"
with open(json_file, 'r') as file:
    estatuto = json.load(file)
estatuto

[{'pages': [{'page': 1,
    'text': '                                                                                                                                                                                         MINISTERIO DA EDUCACAO\n                                                                                                                                                                                           MINISTÉRIO                                                      DA EDUCAÇÃO\n                                                                INSTITUTO FEDERAL DE EDUCACAO, CIENCIA E TECNOLOGIA DE SAO PAULO\n                                                                  INSTITUTO FEDERAL                                                                                        DE EDUCAÇÃO,                                                                   CIÊNCIA                                   E TECNOLOGIA                                                        

### Gerando texto corrido a partir da página 1 e excluindo o rodapé

In [63]:
pages = estatuto[0]["pages"][10:]
pages[0]

{'page': 11,
 'text': '                                      MINISTÉRIO DA EDUCAÇÃO\n\n\n        INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO\n\n\n                       ORGANIZAÇÃO DIDÁTICA DA EDUCAÇÃO BÁSICA\n\n\n    INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO\n\n\n                 (Aprovada pela Resolução Nº 62/2018 - CONSUP/IFSP, de 07/08/2018)\n                (Alterada pela Resolução Normativa IFSP nº 03, de 03 de agosto de 2021).\n              (Alterada pela Resolução Normativa IFSP nº 06, de 09 de novembro de 2021).\n                 (Alterada pela Resolução Normativa nº 10, de 06 de setembro de 2022).\n\n\n                                                TÍTULO I\n\n\n                                  DAS DIRETRIZES GERAIS\n\n\n                                               CAPÍTULO I\n\n\n                                 DA NATUREZA E FINALIDADES\n\n\nArt. 1º. O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE\nSÃO PAULO (IFSP), 

In [67]:
pages = estatuto[0]["pages"][10:]
dados = []
#pattern = re.compile(r'(\n*\s*(Aprovado pela.*))', re.MULTILINE) # Estatuto


for page in pages:
    text = page['text'].strip()

    # Estatuto
    #text = pattern.split(text)[0]+"\n"

    # Organização Didática - Graduação
    # Removendo número da página
    #pattern = r'^\d+\s*'
    #text = re.sub(pattern, '', text, flags=re.MULTILINE)

    # Organização Didática - Ensino Básico
    # Removendo rodapé
    pattern = r'(?i)Alterad[ao] pela Resolução Normativa(?: IFSP)? nº \d{2}, de \d{2} de \w+ de \d{4}\.\s*'
    text = re.sub(pattern, '', text)

    dados.append(text)


dados

['MINISTÉRIO DA EDUCAÇÃO\n\n\n        INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO\n\n\n                       ORGANIZAÇÃO DIDÁTICA DA EDUCAÇÃO BÁSICA\n\n\n    INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO\n\n\n                 (Aprovada pela Resolução Nº 62/2018 - CONSUP/IFSP, de 07/08/2018)\n                (Alterada pela Resolução Normativa IFSP nº 03, de 03 de agosto de 2021).\n              (Alterada pela Resolução Normativa IFSP nº 06, de 09 de novembro de 2021).\n                 (Alterada pela Resolução Normativa nº 10, de 06 de setembro de 2022).\n\n\n                                                TÍTULO I\n\n\n                                  DAS DIRETRIZES GERAIS\n\n\n                                               CAPÍTULO I\n\n\n                                 DA NATUREZA E FINALIDADES\n\n\nArt. 1º. O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE\nSÃO PAULO (IFSP), constituído mediante transformação do Centro Federal de\nEd

### Adicionando marcadores para indicar inicio/término de Título/Capítulo

In [68]:
texto_completo = ''.join(dados)
texto_completo

'MINISTÉRIO DA EDUCAÇÃO\n\n\n        INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO\n\n\n                       ORGANIZAÇÃO DIDÁTICA DA EDUCAÇÃO BÁSICA\n\n\n    INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO\n\n\n                 (Aprovada pela Resolução Nº 62/2018 - CONSUP/IFSP, de 07/08/2018)\n                (Alterada pela Resolução Normativa IFSP nº 03, de 03 de agosto de 2021).\n              (Alterada pela Resolução Normativa IFSP nº 06, de 09 de novembro de 2021).\n                 (Alterada pela Resolução Normativa nº 10, de 06 de setembro de 2022).\n\n\n                                                TÍTULO I\n\n\n                                  DAS DIRETRIZES GERAIS\n\n\n                                               CAPÍTULO I\n\n\n                                 DA NATUREZA E FINALIDADES\n\n\nArt. 1º. O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE\nSÃO PAULO (IFSP), constituído mediante transformação do Centro Federal de\nEdu

In [69]:
import re

# Expressão regular para encontrar linhas que começam com 'Título'
pattern_titulo = r'^\n*\s*(TÍTULO.*)'

# Função para adicionar a marcação antes de linhas que começam com 'Título'
def adicionar_marcacao_titulo(match):
    return '\n>>>>>\n\n' + match.group(1)

texto_marcado = re.sub(pattern_titulo, adicionar_marcacao_titulo, texto_completo, flags=re.MULTILINE)


pattern_capitulo = r'^\n*\s*(CAPÍTULO.*)'

def adicionar_marcacao_capitulo(match):
    return '\n*****\n\n' + match.group(1)

texto_marcado = re.sub(pattern_capitulo, adicionar_marcacao_capitulo, texto_marcado, flags=re.MULTILINE)

print(texto_marcado[:300])

MINISTÉRIO DA EDUCAÇÃO


        INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO


                       ORGANIZAÇÃO DIDÁTICA DA EDUCAÇÃO BÁSICA


    INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO


                 (Aprovada pela Resolução Nº 62/2018 - CONSUP/IF


### Colocando o título dos capítulos em uma única linha

In [70]:
# Dividir o texto em linhas
linhas = texto_marcado.split('\n')

# Inicialização de variáveis
agrupando_capitulo = False
capitulo_atual = ""
novo_texto = []

# Regex patterns
#capitulo_pattern = re.compile(r'^\s*CAPÍTULO.*') # Estatuto
capitulo_pattern = re.compile(r'^\s*[TÍTULO|CAPÍTULO].*')
separador_pattern = re.compile(r'^\s*(SEÇÃO|Art\.)')

for linha in linhas:

    if capitulo_pattern.match(linha):
        # Encontrou uma linha que começa com "CAPÍTULO"
        if agrupando_capitulo:
            novo_texto.append(capitulo_atual.strip())
        capitulo_atual = f"{linha.strip()} -"
        agrupando_capitulo = True

    elif separador_pattern.match(linha):
        # Encontrou uma linha que começa com "SEÇÃO" ou "Art."
        if agrupando_capitulo:
            novo_texto.append(capitulo_atual.strip()+". \n")
            agrupando_capitulo = False
        novo_texto.append(linha.strip())

    else:
        # Parte do conteúdo do capítulo
        if agrupando_capitulo:
            capitulo_atual += ' ' + linha.strip()
        else:
            novo_texto.append(linha.strip())

# Adiciona o último capítulo se ainda estiver agrupando
if agrupando_capitulo:
    novo_texto.append(capitulo_atual.strip()+"\n")

# Juntar as linhas em um novo texto
texto_marcado = '\n'.join(novo_texto)
print(texto_marcado[:300])



MINISTÉRIO DA EDUCAÇÃO


INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO


ORGANIZAÇÃO DIDÁTICA DA EDUCAÇÃO BÁSICA -   INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO   (Aprovada pela Resolução Nº 62/2018 - CONSUP/IFSP, de 07/08/2018) (Alterada pela Resolução Normat


### Montando JSON agrupando artigos por capítulo/título



```
{
 titulo: Estatuto
 texto_completo: texto completo extraido do PDF
 partes:[
    {titulo: Título,
     capítulos: [
        {titulo: Título do capítulo,
         artigos: texto com todos os artigos do capítulo
        }
     ]
    },
    {titulo: Título,
     artigos: texto com todos os artigos do título
    }
 ]
}
```




In [71]:
texto_marcado

'MINISTÉRIO DA EDUCAÇÃO\n\n\nINSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO\n\n\nORGANIZAÇÃO DIDÁTICA DA EDUCAÇÃO BÁSICA -   INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO   (Aprovada pela Resolução Nº 62/2018 - CONSUP/IFSP, de 07/08/2018) (Alterada pela Resolução Normativa IFSP nº 03, de 03 de agosto de 2021). (Alterada pela Resolução Normativa IFSP nº 06, de 09 de novembro de 2021). (Alterada pela Resolução Normativa nº 10, de 06 de setembro de 2022).  >>>>>\nTÍTULO I -   DAS DIRETRIZES GERAIS  *****\nCAPÍTULO I -   DA NATUREZA E FINALIDADES\nArt. 1º. O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE - SÃO PAULO (IFSP), constituído mediante transformação do Centro Federal de Educação Tecnológica de São Paulo, nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, possui natureza jurídica de autarquia, vinculada ao Ministério da Educação (MEC), detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar

In [72]:
texto_sem_espacos = texto_marcado
# Estatuto
#texto_sem_espacos = texto_sem_espacos.replace("\n\n>>>>>\n\n", ". ")
#texto_sem_espacos = texto_sem_espacos.replace("\n\n*****\n\n", ". ")

texto_sem_espacos = texto_sem_espacos.replace(">>>>>", ". ")
texto_sem_espacos = texto_sem_espacos.replace("*****", ". ")

texto_sem_espacos = re.sub(r'\n', ' ', texto_sem_espacos)
texto_sem_espacos = re.sub(r'\s+\.\s+', '. ', texto_sem_espacos)
texto_sem_espacos = re.sub(r'\.\.', '. ', texto_sem_espacos)
texto_sem_espacos = re.sub(r'Art.', 'Art', texto_sem_espacos)
texto_sem_espacos = re.sub(r' ', '', texto_sem_espacos)

texto_sem_espacos = re.sub(r'\s+', ' ', texto_sem_espacos)
texto_sem_espacos

'MINISTÉRIO DA EDUCAÇÃO INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO ORGANIZAÇÃO DIDÁTICA DA EDUCAÇÃO BÁSICA - INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO (Aprovada pela Resolução Nº 62/2018 - CONSUP/IFSP, de 07/08/2018) (Alterada pela Resolução Normativa IFSP nº 03, de 03 de agosto de 2021). (Alterada pela Resolução Normativa IFSP nº 06, de 09 de novembro de 2021). (Alterada pela Resolução Normativa nº 10, de 06 de setembro de 2022). TÍTULO I - DAS DIRETRIZES GERAIS. CAPÍTULO I - DA NATUREZA E FINALIDADES Art 1º. O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE - SÃO PAULO (IFSP), constituído mediante transformação do Centro Federal de Educação Tecnológica de São Paulo, nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, possui natureza jurídica de autarquia, vinculada ao Ministério da Educação (MEC), detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar, conforme disposto em seu Estatuto

In [74]:
doc_json = {}
doc_json['titulo'] = 'Organização Didática da Educação Básica'
doc_json['texto_completo'] = texto_completo
doc_json['texto_compacto'] = texto_sem_espacos
doc_json['partes'] = []
titulos = texto_marcado.split('\n\n>>>>>\n\n')
for titulo in titulos:
    titulos_json = {}
    titulo_partes = titulo.split('\n*****\n\n')

    # Se tiver capítulos, colocar os artigos junto de seus capítulos
    if len(titulo_partes) != 1:
        capitulos = titulo_partes[1:]
        titulos_json['titulo'] = titulo_partes[0].replace('\n','')
        titulos_json['capitulos'] = []
        for capitulo in capitulos:
            capitulo_partes = capitulo.split('\n\n')
            artigos = ' '.join(capitulo_partes[1:])
            titulos_json['capitulos'].append({'titulo': capitulo_partes[0].strip(),'artigos': artigos})
    else:
        # Titulos sem capítulos
        titulo_partes = titulo_partes[0].split('\n\n\n')
        titulos_json['titulo']  = titulo_partes[0].replace('\n','')
        artigos                 = ' '.join(titulo_partes[1:])
        titulos_json['artigos'] = artigos


    if len(titulos_json) > 0:
        doc_json['partes'].append(titulos_json)

doc_json['partes']

[{'titulo': 'MINISTÉRIO DA EDUCAÇÃO',
  'artigos': 'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO ORGANIZAÇÃO DIDÁTICA DA EDUCAÇÃO BÁSICA -   INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO   (Aprovada pela Resolução Nº 62/2018 - CONSUP/IFSP, de 07/08/2018) (Alterada pela Resolução Normativa IFSP nº 03, de 03 de agosto de 2021). (Alterada pela Resolução Normativa IFSP nº 06, de 09 de novembro de 2021). (Alterada pela Resolução Normativa nº 10, de 06 de setembro de 2022).  >>>>>\nTÍTULO I -   DAS DIRETRIZES GERAIS  *****\nCAPÍTULO I -   DA NATUREZA E FINALIDADES\nArt. 1º. O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE - SÃO PAULO (IFSP), constituído mediante transformação do Centro Federal de Educação Tecnológica de São Paulo, nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, possui natureza jurídica de autarquia, vinculada ao Ministério da Educação (MEC), detentora de autonomia administrativa, patrimonial, financeira, didático-pedagó

In [75]:
with open('/content/ia024-projeto-rag/2-dataset/organizacao-didatica-educacao-basica.json', 'w', encoding='utf-8') as f:
    json.dump(doc_json, f, ensure_ascii=False, indent=4)

In [76]:
print("push to git...")
msg = 'Adicionando JSON da Organização Didática da Educação Básica processado'
save_on_git(msg)

push to git...
/content/ia024-projeto-rag
From https://github.com/rsabilio/ia024-projeto-rag
 * branch            main       -> FETCH_HEAD
Already up to date.
[main 469b786] Adicionando JSON da Organização Didática da Educação Básica processado
 1 file changed, 11 insertions(+)
 create mode 100644 2-dataset/organizacao-didatica-educacao-basica.json
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 116.28 KiB | 3.63 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/rsabilio/ia024-projeto-rag
   ab09f8c..469b786  main -> main


## Portarias

### Testes

In [ ]:
json_file = f"{src_directory}/2014/03 - Março/Mar_Port_1324_NDE_BTV.pdf.json"
with open(json_file, 'r') as file:
    portaria = json.load(file)
portaria

In [76]:
pages = portaria[0]["pages"]
dados = []

pattern = r'(?i)\bPORTARIA\b\s*(?:n\s*)?\d{1,3}(?:\.\d{3})*(?:,\s*|\s+)DE?\s*\d{1,2}\s*(?:DE\s*)?\w+\s*DE?\s*\d{4}'
titulo = ''
for page in pages:

    for item in page['items']:
        text = item['value'].strip()
        text = re.sub(r'n"', 'n', text)
        print(text)
        matches = re.findall(pattern, text)
        print(matches)
        print('----')
        if len(matches) > 0 and titulo == '':
            titulo = matches[0]

        dados.append(text)

print(f"Título: {titulo}")

#print('. '.join(dados))

INSTITUTO FEDERAL DE MINISTERIO DA EDUCACAO EDUCACAO, CIENCIA E TECNOLOGIA DE SAO PAULO
[]
----
Portaria n 1.324, de 26 marco de 2014

O DIRETOR GERAL DO CAMPUS BOITUVA DO INSTITUTO FEDERAL DE uso de suas atribuicoes legais, e considerando o que consta na Portaria 2.618, de 28 de agosto de 2012 e o Memorando 03/2014-BTV-CDS de 26 de marco de 2014

RESOLVE:

Art: 1" REVOGAR, Portaria n92.826 de 10 de junho de 2013, que trata do Nucleo Docente Estruturante do Curso de Tecnologia de Analise Desenvolvimento de Sistemas do Campus Boituva.

Art: 20 DESIGNAR, os servidores abaixo relacionados para, sob coordenacao presidencia do primeiro, constituirem Nucleo Docente Estruturante do Curso de Tecnologia de Analise Desenvolvimento de Sistemas do Campus Boituva.

Fernando Vernal Salina
Giorjety Licorini Dias
Francisco Cesar de Oliveira
Thiago Pedro Donadon Homem
Felipe Nunes Gaia

Art: 30 Esta portaria entra em vigor nesta data.

BRUNO NOGUEIRA LUZ

Publicado no Quadro de Avisos do Campus BTV em:

In [115]:
pattern = r'(?i)\bPORTARIA\b\s*(?:n[\.]?[oºS\"]?\s*[.\d]?)?\s*\d{1,3}(?:\.\d{3})*(?:,\s*|\s+)DE?\s*\d{1,2}\s*(?:DE\s*)?\w+\s*DE?\s*\d{4}'

t = """
PORTARIA 1.024 DE 11 DE MARCO DE 2014
Portaria n" 1.324, de 26 marco de 2014
PORTARIA N.S1.600 de 09 de Abril de 2014.
PORTARIA No 1.495 DE 04 DE ABRIL DE 2014.
"""
#print(t)
matches = re.findall(pattern, t)
print(matches)


['PORTARIA 1.024 DE 11 DE MARCO DE 2014', 'Portaria n" 1.324, de 26 marco de 2014', 'PORTARIA N.S1.600 de 09 de Abril de 2014', 'PORTARIA No 1.495 DE 04 DE ABRIL DE 2014']


In [119]:
pattern_novo   = r'(?i)\bPORTARIA\b\s+[A-Z]+\.\d{1,4}/\d{4},\s+DE\s+\d{1,2}\s+DE\s+\w+\s+DE\s+\d{4}'

t = """
PORTARIA BTV.0001/2014, DE 24 DE OUTUBRO DE 2014
"""

matches = re.findall(pattern_novo, t)
print(matches)

['PORTARIA BTV.0001/2014, DE 24 DE OUTUBRO DE 2014']


### Processando

In [17]:
def preprocess_and_save_json(campus, year, src_dir, dst_dir):
    src_dir = f"{src_dir}/{year}"
    dst_dir = f"{dst_dir}/"

    pattern_antigo = r'(?i)\bPORTARIA\b\s*(?:n[\.]?[oºS\"]?\s*[.\d]?)?\s*\d{1,3}(?:\.\d{3})*(?:,\s*|\s+)DE?\s*\d{1,2}\s*(?:DE\s*)?\w+\s*DE?\s*\d{4}'
    pattern_novo   = r'(?i)\bPORTARIA\b\s+[A-Z]+\.\d{1,4}/\d{4},\s+DE\s+\d{1,2}\s+DE\s+\w+\s+DE\s+\d{4}'

    json_objs = []
    for root, dirs, files in os.walk(src_dir):
        for file in files:
            src_file_path  = os.path.join(root, file)
            relative_path = os.path.relpath(root, src_dir)
            #dst_file_dir  = os.path.join(dst_dir, relative_path)
            dst_file_dir   = dst_dir
            dst_file_path  = os.path.join(dst_dir, f"{campus}-{year}.json")

            # Cria o diretório de destino se não existir
            os.makedirs(dst_file_dir, exist_ok=True)

            try:
                print(f"{src_file_path}")
                with open(src_file_path, 'r') as file:
                    portaria = json.load(file)

                if len(portaria) > 0:
                    pages = portaria[0]["pages"]
                    dados = []

                    titulo = ''
                    ano    = year
                    mes    = relative_path.split(' - ')[0]
                    for page in pages:

                        for item in page['items']:

                            text = ''
                            if item['type'] == 'table':
                                text = item['md']
                            else:
                                text = item['value'].strip()

                            # Procurar o nome da portaria no padrao antigo
                            matches = re.findall(pattern_antigo, text)
                            if len(matches) > 0 and titulo == '':
                                titulo = matches[0]
                                print(f"{src_file_path}")
                                print(f"Titulo Antigo: {titulo}\n")

                            # Percebi que em Out/2014 o nome das portarias mudou de padrão
                            # Priorizar o padrão novo
                            matches = re.findall(pattern_novo, text)
                            if len(matches) > 0:
                                titulo = matches[0]
                                print(f"{src_file_path}")
                                print(f"Titulo Novo: {titulo}\n")

                            dados.append(text)


                        texto_completo = '. '.join(dados)
                        texto_completo = texto_completo.replace('.. ', '. ')
                        texto_completo = re.sub(r'Art.', 'Art', texto_completo)
                        texto_completo = re.sub(r'\s+', ' ', texto_completo)

                    if titulo != '':
                        json_objs.append({
                                        'campus': campus
                                        ,'year': ano
                                        ,'month': mes
                                        ,'title:':titulo
                                        ,'texto_completo':texto_completo
                                        })
                else:
                    raise Exception('Arquivo vazio')
            except Exception as e:
                print(f"Erro ao processar o arquivo {src_file_path}: {e}")

    if len(json_objs) > 0:
        with open(dst_file_path, 'w', encoding='utf-8') as f:
            json.dump(json_objs, f, ensure_ascii=False, indent=4)


update_local_repo()

# Caminhos
campus = "boituva"
src_directory = f"{root}/1-files-content/2-{campus}/portarias"
dst_directory = f"{root}/2-dataset/2-{campus}/portarias"


# Extrai e salva o conteúdo mantendo a estrutura dos diretórios
for ano in range(2020, 2025):
    print(f"\nProcessing: {ano}")

    preprocess_and_save_json(campus, ano, src_directory, dst_directory)

    print("\npushing to git...")
    msg = f'Adicionando dataset Portarias de Boituva - {ano}'
    save_on_git(msg)



/content/ia024-projeto-rag
From https://github.com/rsabilio/ia024-projeto-rag
 * branch            main       -> FETCH_HEAD
Already up to date.

Processing: 2020
/content/ia024-projeto-rag/1-files-content/2-boituva/portarias/2020/03 - Março/Mar_Port_0015_Substituição da CBI por motivo de férias_BTV.pdf.json
Erro ao processar o arquivo /content/ia024-projeto-rag/1-files-content/2-boituva/portarias/2020/03 - Março/Mar_Port_0015_Substituição da CBI por motivo de férias_BTV.pdf.json: Arquivo vazio
/content/ia024-projeto-rag/1-files-content/2-boituva/portarias/2020/03 - Março/Mar_Port_0012_Designar servidor comissão de seleção de discentes SAKURA_BTV.pdf.json
Erro ao processar o arquivo /content/ia024-projeto-rag/1-files-content/2-boituva/portarias/2020/03 - Março/Mar_Port_0012_Designar servidor comissão de seleção de discentes SAKURA_BTV.pdf.json: Arquivo vazio
/content/ia024-projeto-rag/1-files-content/2-boituva/portarias/2020/03 - Março/Mar_Port_0011_Designar servidores para Comissão de 